In [98]:
import pandas as pd
import os

ROOT_DIR = os.path.abspath("")
DATA_DIR = os.path.join(ROOT_DIR, "data")

zip_tract_mapping_df = pd.read_excel(
    os.path.join(DATA_DIR, "ZIP_TRACT_122020 - Denver Only.xlsx"),
    sheet_name= "Denver ZIP and Full Code Tracts"
)

unique_tracts = zip_tract_mapping_df["TRACT"].unique()
unique_zips = zip_tract_mapping_df["ZIP"].unique()

income_df = pd.read_csv(os.path.join(DATA_DIR, "raw", "Income - All Counties.csv")).transpose()
home_own_rent_df = pd.read_csv(os.path.join(DATA_DIR, "raw", "Own v Rent - All Counties.csv")).transpose()
race_df = pd.read_csv(os.path.join(DATA_DIR, "raw", "Race - All Counties.csv")).transpose()
educational_attainment_df = pd.read_csv(os.path.join(DATA_DIR, "raw", "acs_educational_attainment.csv")).drop('Unnamed: 1538', axis='columns')
age_df = pd.read_csv(os.path.join(DATA_DIR, "raw", "acs_age.csv")).drop('Unnamed: 914', axis='columns')

CENSUS_TRACT_COLNAME = 'census_tract'
CENSUS_TRACT_NAME_COLNAME_DECENNIAL = 'Label (Grouping)'
CENSUS_TRACT_NAME_COLNAME_ACS = 'Geographic Area Name'

/var/folders/_d/lrwnsf417_n1mbkjfjb9grt99pg3kq/T/ipykernel_22917/2478060002.py:18: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,

In [99]:
age_df

,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C01_001MA,S0101_C01_001EA,S0101_C01_002E,S0101_C01_002EA,S0101_C01_002M,S0101_C01_002MA,...,S0101_C06_036MA,S0101_C06_037E,S0101_C06_037EA,S0101_C06_037M,S0101_C06_037MA,S0101_C06_038E,S0101_C06_038EA,S0101_C06_038M,S0101_C06_038MA,Unnamed: 914
0,Geography,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Annotation of Margin of Error!!Total!!Total po...,Annotation of Estimate!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under ...,Annotation of Estimate!!Total!!Total populatio...,Margin of Error!!Total!!Total population!!AGE!...,Annotation of Margin of Error!!Total!!Total po...,...,Annotation of Margin of Error!!Percent Female!...,Estimate!!Percent Female!!Total population!!PE...,Annotation of Estimate!!Percent Female!!Total ...,Margin of Error!!Percent Female!!Total populat...,Annotation of Margin of Error!!Percent Female!...,Estimate!!Percent Female!!Total population!!PE...,Annotation of Estimate!!Percent Female!!Total ...,Margin of Error!!Percent Female!!Total populat...,Annotation of Margin of Error!!Percent Female!...,NaN
1,1400000US08001007801,"Census Tract 78.01, Adams County, Colorado",4274,647,NaN,NaN,301,NaN,105,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
2,1400000US08001007802,"Census Tract 78.02, Adams County, Colorado",4491,767,NaN,NaN,353,NaN,213,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
3,1400000US08001007900,"Census Tract 79, Adams County, Colorado",5561,694,NaN,NaN,337,NaN,167,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
4,1400000US08001008000,"Census Tract 80, Adams County, Colorado",5713,992,NaN,NaN,442,NaN,167,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,1400000US08059060501,"Census Tract 605.01, Jefferson County, Colorado",8098,839,NaN,NaN,406,NaN,155,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
589,1400000US08059980000,"Census Tract 9800, Jefferson County, Colorado",0,12,NaN,NaN,0,NaN,12,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
590,1400000US08059980400,"Census Tract 9804, Jefferson County, Colorado",1325,369,NaN,NaN,0,NaN,12,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN
591,1400000US08059980700,"Census Tract 9807, Jefferson County, Colorado",0,12,NaN,NaN,0,NaN,12,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),NaN


In [90]:
COUNTY_CENSUS_MAPPING = {
    'Adams County' : '001',
    'Arapahoe County' : '005',
    'Denver County' : '031',
    'Jefferson County' : '059'
}

def find_census_tract_number(tract_label):
    
    census_name_elems = [substr.strip() for substr in tract_label.split(",")]
    try:
        county = census_name_elems[1]
        county_tract = COUNTY_CENSUS_MAPPING[county]
        census_tract_name = census_name_elems[0]
        census_tract_number = census_tract_name.lstrip("Census Tract ").replace(".", "")
        census_tract_number = census_tract_number.ljust(4, "0").zfill(6)
    
        full_tract = f"08{county_tract}{census_tract_number}"
    except IndexError:
        return None
    
    return full_tract

In [94]:
def assign_columns(df, col_idx = 0):
    
    df.columns = df.iloc[col_idx]
    df.drop(df.index[col_idx], inplace=True)

    return df

def remove_unicode_from_column_names(colname):

    return str(colname).replace(u'\xa0', u'')

home_own_rent_df = assign_columns(home_own_rent_df)
race_df = assign_columns(race_df)
income_df = assign_columns(income_df)
educational_attainment_df = assign_columns(educational_attainment_df)
age_df = assign_columns(age_df)

home_own_rent_df.columns = [remove_unicode_from_column_names(colname) for colname in home_own_rent_df.columns]
race_df.columns = [remove_unicode_from_column_names(colname) for colname in race_df.columns]
income_df.columns = [remove_unicode_from_column_names(colname) for colname in income_df.columns]


In [96]:
home_own_rent_df[CENSUS_TRACT_COLNAME] = home_own_rent_df.apply(lambda x: find_census_tract_number(x.name), axis = 1)
race_df[CENSUS_TRACT_COLNAME] = race_df.apply(lambda x: find_census_tract_number(x.name), axis = 1)
income_df[CENSUS_TRACT_COLNAME] = income_df.apply(lambda x: find_census_tract_number(x.name), axis = 1)
educational_attainment_df[CENSUS_TRACT_COLNAME] = educational_attainment_df[CENSUS_TRACT_NAME_COLNAME_ACS].apply(lambda x: find_census_tract_number(x))
age_df[CENSUS_TRACT_COLNAME] = age_df[CENSUS_TRACT_NAME_COLNAME_ACS].apply(lambda x: find_census_tract_number(x))

In [97]:
age_df

,Geography,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Annotation of Margin of Error!!Total!!Total population,Annotation of Estimate!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under 5 years,Annotation of Estimate!!Total!!Total population!!AGE!!Under 5 years,Margin of Error!!Total!!Total population!!AGE!!Under 5 years,Annotation of Margin of Error!!Total!!Total population!!AGE!!Under 5 years,...,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Annotation of Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Annotation of Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Sex,Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,Annotation of Estimate!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,Annotation of Margin of Error!!Percent Female!!Total population!!PERCENT ALLOCATED!!Age,NaN,census_tract
1,1400000US08001007801,"Census Tract 78.01, Adams County, Colorado",4274,647,NaN,NaN,301,NaN,105,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08001007801
2,1400000US08001007802,"Census Tract 78.02, Adams County, Colorado",4491,767,NaN,NaN,353,NaN,213,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08001007802
3,1400000US08001007900,"Census Tract 79, Adams County, Colorado",5561,694,NaN,NaN,337,NaN,167,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08001007900
4,1400000US08001008000,"Census Tract 80, Adams County, Colorado",5713,992,NaN,NaN,442,NaN,167,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08001008000
5,1400000US08001008100,"Census Tract 81, Adams County, Colorado",1440,193,NaN,NaN,33,NaN,36,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08001008100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,1400000US08059060501,"Census Tract 605.01, Jefferson County, Colorado",8098,839,NaN,NaN,406,NaN,155,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08059060501
589,1400000US08059980000,"Census Tract 9800, Jefferson County, Colorado",0,12,NaN,NaN,0,NaN,12,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08059009800
590,1400000US08059980400,"Census Tract 9804, Jefferson County, Colorado",1325,369,NaN,NaN,0,NaN,12,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08059009804
591,1400000US08059980700,"Census Tract 9807, Jefferson County, Colorado",0,12,NaN,NaN,0,NaN,12,NaN,...,(X),(X),(X),(X),(X),(X),(X),(X),NaN,08059009807


In [67]:
educational_attainment_columns = [CENSUS_TRACT_COLNAME] + [col for col in educational_attainment_df.columns if col.startswith('Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over')]
educational_attainment_df = educational_attainment_df[educational_attainment_columns]

age_columns = [CENSUS_TRACT_COLNAME] + [col for col in age_df.columns if col.startswith('Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over')]

In [68]:
def transform_counts_into_percentages(df, total_column, list_of_columns_to_transform):

    rows_w_percentages = []

    for idx, row in df.iterrows():

        denominator = int(str(row[total_column]).replace(",", ""))
        
        try:
            for col in list_of_columns_to_transform:
                reformatted_col_name = col.lower().replace(" ","_").replace("$", "").replace(",", "").replace(":", "").replace(",000", "k").replace("!!","_")
                row[f"{reformatted_col_name}_percentage_total"] = int(str(row[col]).replace(",", "")) / denominator

            rows_w_percentages.append(row)

        except ZeroDivisionError:
            continue

    return pd.DataFrame(rows_w_percentages)

In [70]:
income_df_w_percentages = transform_counts_into_percentages(
    income_df,
    'Total:',
    [col for col in income_df.columns if col not in ['Total:', CENSUS_TRACT_COLNAME]]
)

race_df_w_percentages = transform_counts_into_percentages(
    race_df,
    'Population of one race:',
    [col for col in race_df.columns if 'alone' in col]
)

home_own_rent_df_w_percentages = transform_counts_into_percentages(
    home_own_rent_df[['Total:', CENSUS_TRACT_COLNAME, 'Owner occupied:', 'Renter occupied:']],
    'Total:',
    ['Owner occupied:', 'Renter occupied:']
)

educational_attainment_df_w_percentages = transform_counts_into_percentages(
    educational_attainment_df,
    'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over',
    [col for col in educational_attainment_df.columns if col not in [CENSUS_TRACT_COLNAME, 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over']]
)

In [71]:
educational_attainment_df_w_percentages


,census_tract,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,estimate_total_age_by_educational_attainment_population_25_years_and_over_less_than_9th_grade_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_9th_to_12th_grade_no_diploma_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_high_school_graduate_(includes_equivalency)_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_some_college_no_degree_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_associate's_degree_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_bachelor's_degree_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_graduate_or_professional_degree_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_high_school_graduate_or_higher_percentage_total,estimate_total_age_by_educational_attainment_population_25_years_and_over_bachelor's_degree_or_higher_percentage_total
1,08001007801,2299,652,442,509,492,83,97,24,1205,121,0.283602,0.192258,0.221401,0.214006,0.036103,0.042192,0.010439,0.524141,0.052632
2,08001007802,2902,738,436,1080,339,90,135,84,1728,219,0.254307,0.150241,0.372157,0.116816,0.031013,0.046520,0.028946,0.595451,0.075465
3,08001007900,3525,540,394,1148,445,334,423,241,2591,664,0.153191,0.111773,0.325674,0.126241,0.094752,0.120000,0.068369,0.735035,0.188369
4,08001008000,3893,367,323,1801,622,278,298,204,3203,502,0.094272,0.082969,0.462625,0.159774,0.071410,0.076548,0.052402,0.822759,0.128949
5,08001008100,1062,32,28,122,111,91,342,336,1002,678,0.030132,0.026365,0.114878,0.104520,0.085687,0.322034,0.316384,0.943503,0.638418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,08059006030,3340,43,136,690,630,216,1127,498,3161,1625,0.012874,0.040719,0.206587,0.188623,0.064671,0.337425,0.149102,0.946407,0.486527
587,08059006040,2723,76,39,415,844,225,658,466,2608,1124,0.027910,0.014322,0.152405,0.309952,0.082629,0.241645,0.171135,0.957767,0.412780
588,08059060501,5340,85,11,534,1016,204,2005,1485,5244,3490,0.015918,0.002060,0.100000,0.190262,0.038202,0.375468,0.278090,0.982022,0.653558
590,08059009804,1263,61,189,479,332,30,44,128,1013,172,0.048298,0.149644,0.379256,0.262866,0.023753,0.034838,0.101346,0.802059,0.136184


In [75]:
income_df_w_percentages.rename({'Total:':'income_total_count'}, axis=1, inplace=True)
race_df_w_percentages.rename({'Population of one race:':'race_total_count'}, axis=1, inplace=True)
home_own_rent_df_w_percentages.rename({'Total:':'home_own_rent_total'}, axis=1, inplace=True)
educational_attainment_df_w_percentages.rename({'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over' : 'population_25_and_over'}, axis = 1, inplace=True)

home_own_rent_df = home_own_rent_df_w_percentages[['home_own_rent_total', CENSUS_TRACT_COLNAME] 
                               + [col for col in home_own_rent_df_w_percentages.columns if 'percentage_total' in col]] #.to_csv(os.path.join(DATA_DIR, "interim", "home_own_rent.csv"))
income_df = income_df_w_percentages[['income_total_count', CENSUS_TRACT_COLNAME] + 
                        [col for col in income_df_w_percentages.columns if 'percentage_total' in col]] #.to_csv(os.path.join(DATA_DIR, "interim", "income.csv"))
race_df = race_df_w_percentages[['race_total_count', CENSUS_TRACT_COLNAME] + 
                      [col for col in race_df_w_percentages.columns if 'percentage_total' in col]] #.to_csv(os.path.join(DATA_DIR, "interim", "race.csv"))
educational_attainment_df = educational_attainment_df_w_percentages[['population_25_and_over', CENSUS_TRACT_COLNAME] + 
                      [col for col in educational_attainment_df_w_percentages.columns if 'percentage_total' in col]]


In [79]:
# income_df = pd.read_csv(os.path.join(DATA_DIR, "interim", "income.csv"))
# race_df = pd.read_csv(os.path.join(DATA_DIR, "interim", "race.csv"))
# home_own_rent_df = pd.read_csv(os.path.join(DATA_DIR, "interim", "home_own_rent.csv"))

combined_df = pd.merge(income_df, race_df, how='outer', on=CENSUS_TRACT_COLNAME)
combined_df = pd.merge(combined_df, home_own_rent_df, how='outer', on=CENSUS_TRACT_COLNAME)
combined_df = pd.merge(combined_df, educational_attainment_df, how='outer', on=CENSUS_TRACT_COLNAME)
for col in combined_df.columns: 
    if 'percentage_total' in col:
        combined_df[col] = combined_df[col].fillna(combined_df[col].median())

In [81]:
MAX_MIN_DICT = {}
for col in [column for column in combined_df.columns if 'percentage_total' in column]:
    MAX_MIN_DICT[col] = {
        'max' : combined_df[col].max(),
        'min' : combined_df[col].min()
    }

rows_w_standardized_values = []
for idx, row in combined_df.iterrows():
    if row[CENSUS_TRACT_COLNAME] is None:
        continue

    for col in [column for column in combined_df.columns if 'percentage_total' in column]:

        denominator = MAX_MIN_DICT[col]['max'] - MAX_MIN_DICT[col]['min']
        standardized_val = (row[col] - MAX_MIN_DICT[col]['min']) / denominator
        standardized_colname = col.replace("percentage_total", "standardized")
        row[standardized_colname] = standardized_val

    
    rows_w_standardized_values.append(row)



In [85]:
standardized_df = pd.DataFrame(rows_w_standardized_values)
standardized_df.to_csv(os.path.join(DATA_DIR, "processed", "standardized.csv"), index=False)

In [87]:
standardized_df = pd.read_csv(os.path.join(DATA_DIR, "processed", "standardized.csv"))

count    650.000000
mean       0.819409
std        0.182180
min        0.000000
25%        0.764714
50%        0.883823
75%        0.939352
max        1.000000
Name: estimate_total_age_by_educational_attainment_population_25_years_and_over_high_school_graduate_or_higher_standardized, dtype: float64

In [88]:
standardized_df_reduced_columns = standardized_df[
    [CENSUS_TRACT_COLNAME] + [col for col in standardized_df.columns if "_standardized" in col]
]

In [15]:
pca_input_columns = [col for col in standardized_df_reduced_columns.columns if '_standardized' in col]
pca_input_df = standardized_df_reduced_columns[pca_input_columns]
census_tracts = standardized_df_reduced_columns[CENSUS_TRACT_COLNAME]


In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.85)
pca.fit(pca_input_df)

PCA(n_components=0.85)

In [18]:
pca_output_df = pd.DataFrame(pca.transform(pca_input_df))

In [19]:
pca_output_df.columns = [f"x{idx}" for idx in range(len(pca.components_))]

In [22]:
pca_output_df[CENSUS_TRACT_COLNAME] = None
pca_output_df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,census_tract
0,1.225187,-0.149232,0.368477,0.295765,-0.017200,-0.046010,0.216196,0.075602,-0.119034,0.104189,0.509474,0.165538,None
1,1.302083,-0.001440,0.244756,0.268733,-0.192988,-0.111020,0.193172,0.111338,-0.124874,0.167984,-0.043474,0.092288,None
2,0.969652,0.064561,0.062836,0.115468,-0.161904,0.033649,-0.075587,-0.169447,0.097774,0.160711,0.016590,-0.063087,None
3,0.611850,0.261121,-0.000724,0.079005,-0.083322,0.033510,-0.235503,-0.216684,0.190810,0.158696,-0.247446,-0.038649,None
4,0.622973,-0.561071,0.427328,-0.146777,0.022960,-0.040714,0.066930,0.076440,0.219344,0.108046,-0.067912,-0.105341,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,0.121072,-0.407953,0.076673,-0.056382,-0.120624,0.056911,-0.005085,0.019283,-0.014072,-0.021790,-0.019763,0.024556,None
646,0.052439,-0.306938,0.040640,-0.051133,-0.082369,0.042198,-0.006720,0.013653,-0.013948,-0.015917,-0.020361,0.021622,None
647,0.109254,-0.390559,0.070469,-0.055478,-0.114036,0.054377,-0.005367,0.018314,-0.014051,-0.020779,-0.019866,0.024051,None
648,0.274948,-0.634429,0.157460,-0.068150,-0.206391,0.089897,-0.001418,0.031904,-0.014351,-0.034957,-0.018421,0.031136,None


In [29]:
pca_output_df.at[0, CENSUS_TRACT_COLNAME] = "101"

In [34]:
for idx in range(0, len(census_tracts)):
    pca_output_df.at[idx, CENSUS_TRACT_COLNAME] = census_tracts[idx]

In [40]:
pca_output_df["vulnerability_index"] = \
    pca_output_df["x0"] + pca_output_df["x1"] + pca_output_df["x2"] + pca_output_df["x3"] + \
    pca_output_df["x4"] + pca_output_df["x5"] + pca_output_df["x6"] + pca_output_df["x7"] + \
    pca_output_df["x8"] + pca_output_df["x9"] + pca_output_df["x10"] + pca_output_df["x11"] 

In [42]:
pca_output_df.to_csv(os.path.join(DATA_DIR, "final", "vulnerability_index_unverified.csv"))
